In [ ]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('stopwords')

In [ ]:
df = pd.read_csv('fake_job_postings.csv')
df.head(5)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report as clrp
kf = KFold(n_splits=5)

In [ ]:
df_desc = df.description

In [ ]:
lemmatizer = WordNetLemmatizer() 
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
stop = stopwords.words('english')

def lemmatize_text1(text):
    return [lemmatizer.lemmatize(w,pos="v") for w in w_tokenizer.tokenize(text)]
def lemmatize_text2(text):
    return [lemmatizer.lemmatize(w,pos="n") for w in text]

In [ ]:
df_desc.fillna('')

In [ ]:
df_desc= df_desc.str.replace('[^\w\s]','') #remove punctuation
df_desc= df_desc.str.replace('\d+', '') #remove numbers 
df_desc= df_desc.apply(lambda x: " ".join(x.lower() for x in str(x).split())) #lowercase
df_desc= df_desc.apply(lemmatize_text1)
df_desc= df_desc.apply(lemmatize_text2)
df_desc= df_desc.apply(lambda x: " ".join(x for x in x if x not in stop)) #remove stopwords

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(2,3))
xtrain_matrix = vectorizer.fit_transform(df_desc)
ytrain_matrix = df.fraudulent

In [ ]:
from sklearn.svm import LinearSVC
classifier= LinearSVC(random_state=0, tol=1e-5) 
for train_index, test_index in kf.split(xtrain_matrix,ytrain_matrix):
    classifier.fit(xtrain_matrix[train_index],ytrain_matrix[train_index])
    ypred=classifier.predict(xtrain_matrix[test_index])
    ytestt=ytrain_matrix[test_index]
    print(clrp(ytestt,ypred))

In [ ]:
xtrain_matrix.shape

In [ ]:
from xgboost import XGBClassifier
classifier2 = XGBClassifier()
for train_index, test_index in kf.split(xtrain_matrix,ytrain_matrix):
    classifier2.fit(xtrain_matrix[train_index],ytrain_matrix[train_index])
    ypred=classifier2.predict(xtrain_matrix[test_index])
    ytestt=ytrain_matrix[test_index]
    print(clrp(ytestt,ypred))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
classifier2 = DecisionTreeClassifier()
for train_index, test_index in kf.split(xtrain_matrix,ytrain_matrix):
    classifier2.fit(xtrain_matrix[train_index],ytrain_matrix[train_index])
    ypred=classifier2.predict(xtrain_matrix[test_index])
    ytestt=ytrain_matrix[test_index]
    print(clrp(ytestt,ypred))